In [3]:
# ==========================================
# 🚀 高精度版：記号保持・誤変換「きごう」完全防止
# ==========================================

!pip install sudachipy sudachidict_core jaconv wikipedia tqdm

import csv
import re
import time
import wikipedia
from jaconv import kata2hira
from sudachipy import dictionary, tokenizer
from tqdm import tqdm

# ------------------------------------------
# 🔧 SudachiPy 初期化
# ------------------------------------------
tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C

# ------------------------------------------
# 🧹 テキスト整形
# ------------------------------------------
def clean_text(text: str) -> str:
    return text.replace("\n", "").strip()

# ------------------------------------------
# 🈁 ふりがな生成（記号・外国語保持／誤変換防止）
# ------------------------------------------
def get_furigana(text: str) -> str:
    """
    - 漢字だけひらがな化
    - カタカナ、英数字、記号、外国語はそのまま保持
    - 「きごう」などの誤出力を完全防止
    """
    result = []
    for m in tokenizer_obj.tokenize(text, mode):
        surface = m.surface()
        reading = m.reading_form()

        # --- 英数字・カタカナ・キリル・記号 → そのまま保持 ---
        if re.fullmatch(r"[A-Za-z0-9А-Яа-яЁёІіЇїЄєҐґァ-ヴー・ー＝×÷＋−±％°℃§『』「」【】（）()［］｛｝〔〕〈〉《》“”‘’・…‥、。！？：；—〜ー－\-\[\]{}<>#@\$%^&*_+=|\\'\"~`！？　。、\s]+", surface):
            result.append(surface)
            continue

        # --- 「漢字を含む語」だけひらがな変換 ---
        if re.search(r"[一-龯]", surface) and reading and reading != surface:
            hira = kata2hira(reading)
            result.append(hira)
        else:
            # ひらがな・カタカナ・未知語・記号 → そのまま
            result.append(surface)

    return "".join(result)

# ------------------------------------------
# ⚙️ 設定
# ------------------------------------------
wikipedia.set_lang("ja")

TARGET_COUNT = 2000
SAVE_INTERVAL = 100
OUTPUT_FILE = "/content/drive/MyDrive/きれいなデータセット/wikipedia_datakirei6.csv"
SLEEP_TIME = 0.05
MIN_SENT_LEN = 5
MAX_SENT_LEN = 300

# ------------------------------------------
# 🚧 データセット生成
# ------------------------------------------
def main():
    collected = 0
    buffer = []

    with open(OUTPUT_FILE, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["kanji_sentence", "hiragana_sentence"])

    print(f"\n--- データセット生成開始 ---")
    pbar = tqdm(total=TARGET_COUNT)

    while collected < TARGET_COUNT:
        try:
            title = wikipedia.random()
            page = wikipedia.page(title)
            text = clean_text(page.summary)

            if not text:
                continue

            sentences = [s.strip() for s in text.split("。") if s.strip()]
            for s in sentences:
                if len(s) < MIN_SENT_LEN or len(s) > MAX_SENT_LEN:
                    continue

                kanji_sentence = s + "。"
                hira_sentence = get_furigana(kanji_sentence)

                if hira_sentence == kanji_sentence:
                    continue

                buffer.append([kanji_sentence, hira_sentence])
                collected += 1
                pbar.update(1)

                if len(buffer) >= SAVE_INTERVAL:
                    with open(OUTPUT_FILE, "a", newline="", encoding="utf-8") as f:
                        writer = csv.writer(f)
                        writer.writerows(buffer)
                    buffer.clear()
                    print(f"💾 {collected}件まで保存完了")

                if collected >= TARGET_COUNT:
                    break

            time.sleep(SLEEP_TIME)

        except wikipedia.exceptions.DisambiguationError:
            continue
        except Exception:
            continue

    # 残りを保存
    if buffer:
        with open(OUTPUT_FILE, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            writer.writerows(buffer)

    pbar.close()
    print(f"\n✅ 完了！ 合計 {collected} 文を {OUTPUT_FILE} に保存しました。")


if __name__ == "__main__":
    main()


--- データセット生成開始 ---


  5%|▌         | 100/2000 [00:28<03:57,  7.99it/s]

💾 100件まで保存完了


 10%|▉         | 196/2000 [00:54<06:52,  4.37it/s]

💾 200件まで保存完了


...

💾 1800件まで保存完了


 95%|█████████▌| 1900/2000 [09:51<00:35,  2.85it/s]

💾 1900件まで保存完了


100%|██████████| 2000/2000 [10:23<00:00,  3.21it/s]

💾 2000件まで保存完了

✅ 完了！ 合計 2000 文を /content/drive/MyDrive/きれいなデータセット/wikipedia_datakirei6.csv に保存しました。
